<a href="https://colab.research.google.com/github/LuanDavid7/AI_EKT/blob/main/AI_Answer_EKT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CÓDIGO ATUAL**

**Instala Bibliotca faltante**

In [1]:
!pip install python-dotenv

**Código Inicial Final**


Versão com Totos os problemas cadastrados

In [13]:
import requests
import json
import datetime
import re
import os
import pytz
import time
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configurações da API
API_KEY = os.getenv("GROQ_API_KEY", "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")
MODEL = "llama3-70b-8192"
URL = "https://api.groq.com/openai/v1/chat/completions"
MAX_RETRIES = 3  # Número máximo de tentativas
INITIAL_DELAY = 1  # Tempo inicial de espera em segundos

# Configurar fuso horário de Brasília
TIMEZONE = pytz.timezone('America/Sao_Paulo')

# Base de conhecimento da Neoenergia Elektro
CATEGORIAS_PROBLEMAS = [
    {"categoria": "Ocorrência Técnica", "tipo": "Emergencia médica", "subtipo": "EMERGENCIA MEDICA - EQUIPAMENTO SUPORTE A VIDA", "criticidade": "Emergência", "termos": ["Tenho equipamento de suporte à vida", "Estou com respirador", "Minha máquina de oxigênio parou"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Emergencia médica", "subtipo": "EMERGENCIA MEDICA - HOSPITAL COM PROBLEMA", "criticidade": "Emergência", "termos": ["O hospital está sem luz", "Preciso de energia para um hospital"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia - Geral", "criticidade": "Médio", "termos": ["Minha casa está sem luz", "Só aqui em casa que caiu a energia"], "interno": True},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia em Medidor Eletrônico", "criticidade": "Médio", "termos": ["Meu relógio digital está apagado", "O medidor novo não está funcionando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia Geral - Área Amplia", "criticidade": "Alto", "termos": ["Faltou luz na rua", "A vizinhança inteira sem energia", "Apagão"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "FALTA DE FASE", "criticidade": "Médio", "termos": ["Algumas lâmpadas acendem, outras não", "Só um lado da casa tem luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Fase em Medidor Eletrônico", "criticidade": "Médio", "termos": ["Meu medidor está piscando em vermelho", "Aparece um erro no painel do relógio"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "SIGFI FALTA DE ENERGIA FOTOVOLTAICO", "criticidade": "Médio", "termos": ["Meu sistema solar não está gerando energia", "As baterias não estão carregando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "OSCILACÃO DE TENSÃO", "criticidade": "Médio", "termos": ["A luz fica piscando", "ventilador oscilando","Meus aparelhos desligam e ligam sozinhos"], "interno": False},  # Alterado para interno=False
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "TENSÃO ALTA", "criticidade": "Alto", "termos": ["A luz está muito forte", "Meus aparelhos estão queimando"], "interno": False},  # Alterado para interno=False
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "TENSÃO BAIXA", "criticidade": "Médio", "termos": ["A luz está fraca", "Meus aparelhos não funcionam direito"], "interno": False},  # Alterado para interno=False
    {"categoria": "Ocorrência Técnica", "tipo": "Outros", "subtipo": "TRANSFORMADOR", "criticidade": "Alto", "termos": ["O transformador está fazendo barulho", "Tem um cheiro estranho vindo do poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "CHOQUE EM INSTALAÇÃO", "criticidade": "Emergência", "termos": ["Estou tomando choque na torneira", "A geladeira está dando choque"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "ELO FUSIVEL QUEIMADO/CAIDO", "criticidade": "Alto", "termos": ["Tem algo pendurado no poste", "Um dispositivo caiu do transformador"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM MEDIDOR", "criticidade": "Médio", "termos": ["Meu relógio de luz está com problema", "O medidor parece quebrado"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM QUICK-LAG", "criticidade": "Médio", "termos": ["O disjuntor do poste parece queimado", "Algo explodiu na caixa do poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que vem para minha casa está solto", "A conexão com o poste está com problema"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO COM ARVORE NA REDE", "criticidade": "Emergência", "termos": ["Está saindo faísca do fio com a árvore", "A árvore está pegando fogo no fio"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NA REDE PRIMARIA", "criticidade": "Emergência", "termos": ["Está saindo fogo dos fios de alta tensão", "Os cabos grossos estão soltando faíscas"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NA REDE SECUNDARIA", "criticidade": "Alto", "termos": ["O fio da rua está soltando faísca", "Está saindo fogo do cabo de luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NO RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que entra na minha casa está pegando fogo", "Tem faísca na entrada de luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO EQUIPAMENTO", "criticidade": "Alto", "termos": ["Roubaram algo do poste", "Levaram equipamento da rede"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO FIACÃO/PADRÃO", "criticidade": "Alto", "termos": ["Roubaram os fios da minha casa", "Levaram o medidor"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO REDE", "criticidade": "Alto", "termos": ["Roubaram os fios da rua", "Tem cabos cortados no poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "INVERSÃO DE FASE", "criticidade": "Médio", "termos": ["Meu motor está girando ao contrário", "O equipamento trifásico não funciona direito"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "OBJETO ESTRANHO NA REDE PRIMARIA", "criticidade": "Alto", "termos": ["Tem algo preso nos fios de alta tensão", "Um objeto está encostado na rede"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "OBJETO ESTRANHO NA REDE SECUNDARIA", "criticidade": "Médio", "termos": ["Jogaram algo nos fios", "Tem um objeto preso na fiação"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "POSTE PADRÃO QUEBRADO", "criticidade": "Alto", "termos": ["Meu poste padrão está quebrado", "A entrada de luz está torta"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NA REDE PRIMARIA", "criticidade": "Emergência", "termos": ["O fio grosso da alta tensão está partido", "Caiu um cabo de alta tensão"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NA REDE SECUNDARIA", "criticidade": "Emergência", "termos": ["O fio da luz caiu", "Tem um cabo quebrado na rua"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NO RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que vem para minha casa está quebrado", "Caiu o cabo da entrada"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "POSTE ABALROADO COM FIO PARTIDO", "criticidade": "Emergência", "termos": ["Poste Caiu e fio caiu","Alguém bateu no poste e o fio caiu", "Poste quebrado com fios soltos"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "POSTE ABALROADO SEM FIO PARTIDO", "criticidade": "Alto", "termos": ["Poste inclinado","O poste está torto depois do acidente", "Bateram no poste mas os fios estão intactos"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Ligação", "subtipo": "Ligação Nova", "criticidade": "Médio", "termos": ["Preciso de uma nova ligação de energia", "Minha casa ainda não tem luz"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação Normal", "criticidade": "Médio", "termos": ["Paguei a conta e a luz não voltou", "Quero religar a energia"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação de Urgência", "criticidade": "Alto", "termos": ["Preciso da energia hoje mesmo", "Tenho uma pessoa doente em casa"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação por Corte", "criticidade": "Médio", "termos": ["tive a energia cortada", "cortaram minha luz", "desligaram minha energia"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Desligamento progamado", "subtipo": "Desligamento para Manutenção", "criticidade": "Médio", "termos": ["Vai faltar luz aqui?", "Tem alguma manutenção programada?"], "interno": False}
]

def obter_saudacao():
    """Retorna uma saudação baseada no horário de Brasília."""
    try:
        hora_atual = datetime.datetime.now(TIMEZONE).hour
        if 5 <= hora_atual < 12:
            return "Bom dia"
        elif 12 <= hora_atual < 18:
            return "Boa tarde"
        else:
            return "Boa noite"
    except Exception:
        return "Olá"

def mensagem_fechamento():
    """Retorna a mensagem padrão de fechamento."""
    return """Agradecemos seu contato! Sempre que precisar estamos disponíveis nos nossos canais:

💻 Site: https://www.neoenergia.com/web/sp
📱 Aplicativo: Android → https://bit.ly/NELK-Android | IOS → https://bit.ly/IOS-NELK
📞 Teleatendimento: 116
Deficiente auditivo ou de fala | 0800 701 01 55
Atendimento fora da área de concessão | 0800 276 0116
Conte sempre com a gente!"""

def gerar_protocolo():
    """Gera um número de protocolo baseado na data e hora atuais em Brasília."""
    agora = datetime.datetime.now(TIMEZONE)
    return f"{agora.year}{agora.month:02d}{agora.day:02d}{agora.hour:02d}{agora.minute:02d}{agora.second:02d}"

def classificar_problema(mensagem, contexto_adicional=""):
    """Classifica o problema com base na mensagem do cliente e contexto adicional."""
    mensagem = mensagem.lower()
    contexto = contexto_adicional.lower() if contexto_adicional else ""

    # Verificar se é um caso de energia cortada primeiro
    if any(termo in mensagem or termo in contexto for termo in ["energia cortada", "luz cortada", "cortaram", "desligaram"]):
        return next((p for p in CATEGORIAS_PROBLEMAS if p["subtipo"] == "Religação por Corte"), None)

    # Verificar emergências primeiro
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("criticidade") == "Emergência":
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem or termo.lower() in contexto:
                    return problema

    # Casos específicos de níveis de tensão (prioridade alta)
    if "oscila" in mensagem or "oscila" in contexto or "tensão" in mensagem or "tensão" in contexto:
        for problema in CATEGORIAS_PROBLEMAS:
            if problema["tipo"] == "Níveis de tensão":
                for termo in problema.get("termos", []):
                    if termo.lower() in mensagem or termo.lower() in contexto:
                        return problema

    # Caso específico: medidor apagado + disjuntores ok = problema externo
    if ("medidor" in mensagem or "medidor" in contexto) and ("apagado" in mensagem or "apagado" in contexto):
        if ("disjuntor" in contexto or "disjuntores" in contexto) and ("ligado" in contexto or "ok" in contexto):
            return next((p for p in CATEGORIAS_PROBLEMAS if p["subtipo"] == "Falta de Energia - Geral"), None)

    # Verificar se é falta de energia geral em área ampla
    if ("sem energia" in mensagem or "sem luz" in mensagem) and ("vizinh" in mensagem or "rua" in mensagem or "bairro" in mensagem):
        return next((p for p in CATEGORIAS_PROBLEMAS if p["subtipo"] == "Falta de Energia Geral - Área Amplia"), None)

    # Outros casos específicos
    for problema in CATEGORIAS_PROBLEMAS:
        for termo in problema.get("termos", []):
            if termo.lower() in mensagem or termo.lower() in contexto:
                return problema

    # Default para falta de energia geral (quando medidor apagado e disjuntores ok)
    if "sem energia" in mensagem or "sem luz" in mensagem:
        return next((p for p in CATEGORIAS_PROBLEMAS if p["subtipo"] == "Falta de Energia - Geral"), None)

    return None

def criar_contexto_ia(historico=[]):
    """Cria um contexto para a IA com informações sobre como deve atuar."""
    return {
        "role": "system",
        "content": """
Você é o assistente virtual da Neoenergia Elektro. Seu objetivo é identificar problemas de energia elétrica e decidir se deve abrir um chamado técnico.

DIRETRIZES IMPORTANTES:
1. Problemas de níveis de tensão (oscilação, tensão alta/baixa) são sempre responsabilidade da Neoenergia, mesmo que afetem apenas uma residência.
2. Para problemas de oscilação/tensão, pergunte apenas para confirmar o tipo de problema e então ofereça imediatamente abrir o chamado técnico.
3. Analise a mensagem do cliente para identificar o tipo de problema.
4. Determine se o problema é interno (dentro da casa do cliente) ou externo (responsabilidade da Neoenergia).
5. Para problemas de falta de energia internos, primeiro oriente a verificar disjuntores do quadro interno e padrão de energia.
6. Se o cliente relatar casa toda sem energia, pergunte se há LED aceso no medidor.
7. Para problemas internos, oriente a procurar um eletricista particular.
8. Para problemas que são responsabilidade da Neoenergia, confirme se o cliente deseja abrir um chamado técnico.
9. Só gere protocolo APÓS a confirmação do cliente.
10. Em casos de emergência (fios partidos, choques, etc), comece sua resposta com "ATENÇÃO!" e dê instruções de segurança.
11. Seja objetivo, claro e direto - respostas curtas são preferíveis.
12. Se o cliente começar já descrevendo uma causa de problema, busque identificar de que forma isso afeta ele.
13. Faça no máximo duas perguntas por vez.
14. Se o cliente apontar que um defeito externo está provocando a falha na sua casa, não peça que o cliente faça verificações internas.
15. Se o cliente dizer que não fez as verificações nos disjuntores solicitadas ou se negar a fazer alguma verificação nos disjuntores, o chamado não pode ser gerado, pois pode se tratar de uma falha interna e é recomendado procurar um eletricista particular.
16. Se o disjuntor está desarmando sozinho, o problema é interno.
17. Se o cliente diz estar totalmente sem energia, mas o medidor apresenta sinais de funcionamento (LEDs acesos, display exibindo valores), o problema também é interno.
18. Se o cliente diz estar totalmente sem energia e está com todos os disjuntores ligados e o display apresenta sinal de funcionamento normal, o problema é interno e deve ser recomendado procurar um eletricista particular, pois pode haver algum disjuntor com problema que precise de troca.
19. Em caso de furto oriente o cliente primeiramente a corrigir os problemas gerados na instalaçção interna, para que após isso a equipe atue na parte externa
20. Quando o cliente relatar que teve a energia cortada, pergunte imediatamente se já realizou o pagamento dos débitos pendentes.

IMPORTANTE: Se o cliente confirmar que deseja abrir um chamado técnico após apontar indícios de uma falha externa, gere o protocolo IMEDIATAMENTE e não faça mais perguntas sobre o problema.

FLUXO DE ATENDIMENTO:
1. Cumprimento inicial.
2. Identificação do problema.
3. Verificação se é interno ou externo.
4. Para casos internos: orientação (disjuntores, eletricista).
5. Para casos externos: confirmação de abertura de chamado técnico.
6. Geração de protocolo (apenas se confirmado).

ORDEM DOS PROCEDIMENTOS:
1. A saudação é livre e muito simpática.
2. Aguarde o cliente apresentar o problema.
3. Questione e dê as orientações.
4. Analise qual é o problema real.
5. Apenas após isso aponte ao cliente qual é a possível falha.
6. Se ofereça para gerar o chamado assim: "Isso parece ser um problema de {problema['tipo']} - {problema['subtipo']}. Você quer que eu abra um chamado técnico?"
7. SE O CLIENTE CONFIRMAR, GERE O PROTOCOLO. Com base nos seus apontamentos, foi gerado o chamado de número 2025-00001 para solucionar o problema de {problema['tipo']} - {problema['subtipo']}. Não é necessário dizer que o problema é externo, isso é óbvio.
8. Não faça mais perguntas, aguarde o cliente esboçar gratidão e uma despedida, então despeça-se, mas claro não cobre agradecimentos, apenas espere.

Para problemas de emergência, priorize a segurança do cliente antes de qualquer procedimento.

DIRETRIZES IMPORTANTES PARA PROBLEMAS DE TENSÃO:
1. Problemas de níveis de tensão (oscilação, tensão alta/baixa) são sempre responsabilidade da Neoenergia.
2. Para estes casos, faça perguntas investigativas para entender melhor o problema:
   - Para oscilação: "As luzes piscam ou os aparelhos reiniciam sozinhos?","Você instalou novos aparelhos no local?", "Existe alguma empresa nova ou em construção ao seu redor?"
   - Para tensão alta: "As lâmpadas estão queimando com frequência ou muito brilhantes?"
   - Para tensão baixa: "Os aparelhos não estão funcionando na potência normal?","você instalou novos aparelhos no local?", "Existe alguma empresa nova ou em construção ao seu redor?"
3. Após 1-2 perguntas investigativas, ofereça imediatamente abrir o chamado técnico.
4. Nunca sugira que o problema possa ser interno nestes casos.
5. Mantenha o tom profissional e empático.

FLUXO ESPECÍFICO PARA RELIGAÇÃO POR CORTE:
1. Quando o cliente mencionar que teve a energia cortada:
   - Pergunte imediatamente: "Você já realizou o pagamento dos débitos pendentes?"
   - Se o cliente confirmar o pagamento:
     - Abra imediatamente um chamado de religação sem pedir verificações adicionais
     - Gere o protocolo com a mensagem: "Chamado de religação aberto com sucesso. Protocolo: {NÚMERO}. Nossa equipe será acionada para restabelecer a energia."
   - Se o cliente não tiver pago:
     - Oriente sobre a necessidade de quitar os débitos primeiro
     - Não gere protocolo

Árvore de Decisão para Identificação de Problemas Elétricos:
1. O cliente está sem energia?
   - Sim → Avance para 2.
   - Não → Avance para 9.

2. Verificação Inicial (Falta de Energia):
   - Pergunta: "Você já verificou os disjuntores do quadro interno e do padrão de entrada? Eles estão ligados?"
     - Sim (disjuntores ligados) → Avance para 3.
     - Não (disjuntores desligados) → Oriente o cliente a religá-los e teste novamente. Se persistir, avance para 3.
   - Pergunta: "O medidor de energia (relógio) está com o display aceso ou LEDs piscando?"
     - Sim (medidor energizado) → Problema interno (verifique instalações do cliente).
     - Não (medidor apagado) → Avance para 3.

3. Escopo da Falta de Energia:
   - Pergunta: "Seus vizinhos também estão sem energia?"
     - Sim (área ampla afetada) → Falta de Energia Geral (relate à equipe de rede).
     - Não (apenas o cliente) → Avance para 4.

4. Tipo de Instalação:
   - Pergunta: "Você possui equipamento de suporte à vida (respirador, oxigênio) afetado?"
     - Sim → Equipamento Suporte a Vida (urgência máxima).
     - Não → Avance para 5.
   - Pergunta: "O problema é em um hospital ou unidade de saúde?"
     - Sim → Hospital com Problema (urgência alta).
     - Não → Avance para 5.

5. Medidor Eletrônico:
   - Pergunta: "O medidor é eletrônico (digital)? Está mostrando alguma mensagem de erro ou luz vermelha?"
     - Sim → Falta de Fase em Medidor Eletrônico ou Falha em Medidor.
     - Não → Avance para 6.

6. Sistema Fotovoltaico:
   - Pergunta: "Você tem sistema de energia solar? Ele está funcionando?"
     - Sim (não está gerando) → SIGFI Falta de Energia Fotovoltaico.
     - Não → Avance para 7.

7. Verificação de Fases:
   - Pergunta: "Algumas tomadas ou lâmpadas funcionam, enquanto outras não?"
     - Sim → Falta de Fase.
     - Não (totalmente sem energia) → Falta de Energia em Ramal de Serviço ou Fio Partido.

8. Danos Visíveis:
   - Pergunta: "Você vê fios partidos, faíscas ou danos no poste/padrão?"
     - Sim → Fio Partido na Rede Secundária/Primária ou Poste Abalroado. Se houver fogo: Fio Dando Fogo.
     - Não → Relate como Falta de Energia Geral (investigação externa).

9. Problemas com Tensão (cliente COM energia):
   - Pergunta: "Você está enfrentando oscilação, tensão alta ou baixa?"
     - Oscilação → "Luzes piscam ou aparelhos reiniciam?" → Oscilação de Tensão.
     - Tensão Alta → "Lâmpadas queimam frequentemente ou brilham muito?" → Tensão Alta.
     - Tensão Baixa → "Lâmpadas fracas ou aparelhos não ligam?" → Tensão Baixa.
   - Perguntas Adicionais para Tensão:
     - "Instalou algum equipamento novo recentemente?"
     - "Há fábricas ou obras próximas?"

10. Problemas no Transformador ou Rede:
    - Pergunta: "Você observa barulhos, cheiros ou vazamentos no transformador?"
      - Sim → Problema no Transformador.
      - Não → Avance para 11.

11. Choques ou Furtos:
    - Pergunta: "Você está levando choque em aparelhos ou torneiras?"
      - Sim → Choque em Instalação.
      - Não → Avance para 12.
    - Pergunta: "Houve furto de cabos, medidor ou equipamentos?"
      - Sim → Furto de Rede/Fiação.

12. Outros Problemas:
    - Pergunta: "Você vê objetos estranhos na rede (pipas, galhos, fios soltos)?"
      - Sim → Objeto Estranho na Rede.
      - Não → Outros (avaliação manual).

13. Fluxo para Religação/Desligamento:
- Pergunta: "A energia foi cortada por falta de pagamento?"
  - Sim → Verifique se o pagamento já foi realizado:
    - Se já pagou: "Você já realizou o pagamento dos débitos? Poderia confirmar para que possamos solicitar a religação?"
      - Confirmado → Solicite Religação Normal/Urgência
      - Não confirmado → Oriente sobre a necessidade de quitar os débitos primeiro
    - Se não pagou: "Para a religação, é necessário primeiro regularizar os débitos pendentes."
  - Não → Avance para outras verificações

Regras-Chave:
- Problemas de tensão (oscilação, alta/baixa) = sempre responsabilidade da Neoenergia
- Medidor apagado + disjuntores ligados = Problema externo.
- Medidor aceso + sem energia = Problema interno.
- Tensão irregular = Foco em equipamentos novos ou mudanças no entorno.
- Fogo/fiação exposta = Urgência máxima (risco de incêndio).
- Energia cortada → Verificar pagamento antes de qualquer ação.
"""
    }

def detectar_confirmacao(mensagem):
    """Detecta se a mensagem do cliente é uma confirmação para abrir chamado."""
    confirmacoes = ["sim", "quero", "pode abrir", "abra", "preciso", "com certeza", "abre", "gera", "gere"]
    return any(termo in mensagem.lower() for termo in confirmacoes)

def detectar_negacao(mensagem):
    """Detecta se a mensagem do cliente é uma negação para abrir chamado."""
    negacoes = ["não", "nao", "não precisa", "nao precisa", "não quero", "nao quero", "cancela", "desisto"]
    return any(termo in mensagem.lower() for termo in negacoes)

def detectar_despedida(mensagem):
    """Detecta se a mensagem do cliente é uma despedida."""
    despedidas = ["sair", "tchau", "encerrar", "obrigado", "agradeço", "até mais", "ate mais", "adeus"]
    return any(termo in mensagem.lower() for termo in despedidas)

def consultar_ia(mensagem_cliente, historico=[], problema_identificado=None):
    """Consulta a IA para obter resposta ao cliente."""
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    messages = [criar_contexto_ia(historico)]

    # Adicionar histórico da conversa
    if historico:
        messages.extend(historico)

    # Adicionar mensagem atual do cliente
    messages.append({"role": "user", "content": mensagem_cliente})

    data = {
        "model": MODEL,
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 500
    }

    # Implementação de retry com backoff exponencial
    delay = INITIAL_DELAY
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(URL, headers=headers, json=data, timeout=10)

            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]

            elif response.status_code == 429:
                error_data = response.json()
                if "try again in" in error_data.get("error", {}).get("message", ""):
                    wait_time = float(error_data["error"]["message"].split("try again in ")[1].split("s")[0])
                    time.sleep(wait_time + 1)
                    continue

            time.sleep(delay)
            delay *= 2

        except Exception as e:
            print(f"Erro ao consultar IA (tentativa {attempt + 1}): {str(e)}")
            if attempt < MAX_RETRIES - 1:
                time.sleep(delay)
                delay *= 2

    # Fallback inteligente baseado no contexto
    contexto = " ".join([msg["content"] for msg in historico if msg["role"] == "user"])
    problema = classificar_problema(mensagem_cliente, contexto)

    # Caso específico para energia cortada
    if any(termo in mensagem_cliente.lower() for termo in ["energia cortada", "luz cortada", "cortaram", "desligaram"]):
        return "Entendo que sua energia foi cortada. Para verificar a possibilidade de religação, preciso confirmar: você já realizou o pagamento dos débitos pendentes?"

    # Caso específico para problemas de tensão
    if problema and problema["tipo"] == "Níveis de tensão":
        return f"Entendi que você está enfrentando problemas de {problema['subtipo']}. Posso abrir imediatamente um chamado técnico para nossa equipe verificar?"

    if problema:
        if problema["criticidade"] == "Emergência":
            return f"ATENÇÃO! Identificamos um problema de {problema['tipo']} - {problema['subtipo']}. Por segurança, mantenha distância. Posso abrir um chamado de emergência?"
        else:
            return f"Isso parece ser um problema de {problema['tipo']} - {problema['subtipo']}. Deseja que eu abra um chamado técnico?"

    return "Poderia detalhar melhor o problema que está enfrentando?"

def atendimento_cliente():
    """Função principal para iniciar atendimento."""
    print("\n=== Atendimento Neoenergia Elektro ===\n")

    historico = []
    primeira_mensagem = True
    problema_identificado = None
    protocolo_gerado = False
    caso_religacao = False
    caso_tensao = False

    while True:
        if primeira_mensagem:
            saudacao = f"{obter_saudacao()}! Sou o Elielzinho da Neoenergia Elektro. Como posso ajudar?"
            print(f"[Atendente] {saudacao}")
            primeira_mensagem = False
            historico.append({"role": "assistant", "content": saudacao})

        mensagem_cliente = input("\n[Cliente] ").strip()

        if detectar_despedida(mensagem_cliente) and not protocolo_gerado:
            print(f"\n[Atendente] {mensagem_fechamento()}")
            break

        historico.append({"role": "user", "content": mensagem_cliente})

        # Se já temos um problema identificado e o cliente confirmou
        if problema_identificado and detectar_confirmacao(mensagem_cliente) and not protocolo_gerado:
            protocolo = gerar_protocolo()

            if caso_religacao:
                resposta = f"Chamado de religação aberto com sucesso. Protocolo: {protocolo}. Nossa equipe será acionada para restabelecer a energia."
            elif caso_tensao:
                resposta = f"Chamado técnico aberto para verificação de {problema_identificado['subtipo']}. Protocolo: {protocolo}. Nossa equipe será acionada para resolver o problema."
            else:
                resposta = f"Chamado técnico aberto para {problema_identificado['tipo']} - {problema_identificado['subtipo']}. Protocolo: {protocolo}. Nossa equipe será acionada."

            print(f"\n[Atendente] {resposta}")
            historico.append({"role": "assistant", "content": resposta})
            protocolo_gerado = True
            continue

        # Verificar se é um caso de religação por corte
        if not problema_identificado and any(termo in mensagem_cliente.lower() for termo in ["energia cortada", "luz cortada", "cortaram", "desligaram"]):
            problema_identificado = classificar_problema(mensagem_cliente)
            caso_religacao = True
            caso_tensao = False
            resposta = "Entendo que sua energia foi cortada. Para verificar a possibilidade de religação, preciso confirmar: você já realizou o pagamento dos débitos pendentes?"
            print(f"\n[Atendente] {resposta}")
            historico.append({"role": "assistant", "content": resposta})
            continue

        # Se for confirmação de pagamento em caso de religação
        if caso_religacao and detectar_confirmacao(mensagem_cliente):
            protocolo = gerar_protocolo()
            resposta = f"Chamado de religação aberto com sucesso. Protocolo: {protocolo}. Nossa equipe será acionada para restabelecer a energia."
            print(f"\n[Atendente] {resposta}")
            historico.append({"role": "assistant", "content": resposta})
            protocolo_gerado = True
            continue

        # Consultar a IA
        resposta = consultar_ia(mensagem_cliente, historico, problema_identificado)

        # Atualizar problema identificado se necessário
        if "problema de" in resposta.lower() and not problema_identificado:
            problema_identificado = classificar_problema(mensagem_cliente, resposta)
            caso_religacao = False  # Resetar flag de religação se outro problema for identificado

            # Verificar se é um caso de tensão
            if problema_identificado and problema_identificado["tipo"] == "Níveis de tensão":
                caso_tensao = True
            else:
                caso_tensao = False

        print(f"\n[Atendente] {resposta}")
        historico.append({"role": "assistant", "content": resposta})

if __name__ == "__main__":
    atendimento_cliente()


=== Atendimento Neoenergia Elektro ===

[Atendente] Boa tarde! Sou o Elielzinho da Neoenergia Elektro. Como posso ajudar?

[Cliente] estou sem energia

[Atendente] Desculpe pelo incoveniente! Você já verificou os disjuntores do quadro interno e do padrão de entrada? Eles estão ligados? Além disso, o medidor de energia (relógio) está com o display aceso ou LEDs piscando?

[Cliente] não

[Atendente] Seu vizinho também está sem energia?

[Cliente] não sei

[Atendente] Ok, vamos investigar mais. Você instalou algum equipamento novo recentemente? Além disso, há alguma empresa nova ou em construção ao seu redor?

[Cliente] não

[Atendente] Isso parece ser um problema de falta de energia. Você quer que eu abra um chamado técnico?

[Cliente] sim

[Atendente] Com base nos seus apontamentos, foi gerado o chamado de número 2025-00001 para solucionar o problema de falta de energia. Nossa equipe será acionada para restabelecer a energia. Obrigado por ter contato conosco!


KeyboardInterrupt: Interrupted by user

In [14]:
import requests
import json
import datetime
import re
import os
import pytz
import time
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configurações da API
API_KEY = os.getenv("GROQ_API_KEY", "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")
MODEL = "llama3-70b-8192"
URL = "https://api.groq.com/openai/v1/chat/completions"
MAX_RETRIES = 3  # Número máximo de tentativas
INITIAL_DELAY = 1  # Tempo inicial de espera em segundos

# Configurar fuso horário de Brasília
TIMEZONE = pytz.timezone('America/Sao_Paulo')

# Base de conhecimento da Neoenergia Elektro
CATEGORIAS_PROBLEMAS = [
    {"categoria": "Ocorrência Técnica", "tipo": "Emergencia médica", "subtipo": "EMERGENCIA MEDICA - EQUIPAMENTO SUPORTE A VIDA", "criticidade": "Emergência", "termos": ["Tenho equipamento de suporte à vida", "Estou com respirador", "Minha máquina de oxigênio parou"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Emergencia médica", "subtipo": "EMERGENCIA MEDICA - HOSPITAL COM PROBLEMA", "criticidade": "Emergência", "termos": ["O hospital está sem luz", "Preciso de energia para um hospital"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia - Geral", "criticidade": "Médio", "termos": ["Minha casa está sem luz", "Só aqui em casa que caiu a energia"], "interno": True},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia em Medidor Eletrônico", "criticidade": "Médio", "termos": ["Meu relógio digital está apagado", "O medidor novo não está funcionando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia Geral - Área Amplia", "criticidade": "Alto", "termos": ["Faltou luz na rua", "A vizinhança inteira sem energia", "Apagão"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "FALTA DE FASE", "criticidade": "Médio", "termos": ["Algumas lâmpadas acendem, outras não", "Só um lado da casa tem luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Fase em Medidor Eletrônico", "criticidade": "Médio", "termos": ["Meu medidor está piscando em vermelho", "Aparece um erro no painel do relógio"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "SIGFI FALTA DE ENERGIA FOTOVOLTAICO", "criticidade": "Médio", "termos": ["Meu sistema solar não está gerando energia", "As baterias não estão carregando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "OSCILACÃO DE TENSÃO", "criticidade": "Médio", "termos": ["A luz fica piscando", "ventilador oscilando","Meus aparelhos desligam e ligam sozinhos"], "interno": False},  # Alterado para interno=False
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "TENSÃO ALTA", "criticidade": "Alto", "termos": ["A luz está muito forte", "Meus aparelhos estão queimando"], "interno": False},  # Alterado para interno=False
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "TENSÃO BAIXA", "criticidade": "Médio", "termos": ["A luz está fraca", "Meus aparelhos não funcionam direito"], "interno": False},  # Alterado para interno=False
    {"categoria": "Ocorrência Técnica", "tipo": "Outros", "subtipo": "TRANSFORMADOR", "criticidade": "Alto", "termos": ["O transformador está fazendo barulho", "Tem um cheiro estranho vindo do poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "CHOQUE EM INSTALAÇÃO", "criticidade": "Emergência", "termos": ["Estou tomando choque na torneira", "A geladeira está dando choque"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "ELO FUSIVEL QUEIMADO/CAIDO", "criticidade": "Alto", "termos": ["Tem algo pendurado no poste", "Um dispositivo caiu do transformador"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM MEDIDOR", "criticidade": "Médio", "termos": ["Meu relógio de luz está com problema", "O medidor parece quebrado"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM QUICK-LAG", "criticidade": "Médio", "termos": ["O disjuntor do poste parece queimado", "Algo explodiu na caixa do poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que vem para minha casa está solto", "A conexão com o poste está com problema"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO COM ARVORE NA REDE", "criticidade": "Emergência", "termos": ["Está saindo faísca do fio com a árvore", "A árvore está pegando fogo no fio"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NA REDE PRIMARIA", "criticidade": "Emergência", "termos": ["Está saindo fogo dos fios de alta tensão", "Os cabos grossos estão soltando faíscas"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NA REDE SECUNDARIA", "criticidade": "Alto", "termos": ["O fio da rua está soltando faísca", "Está saindo fogo do cabo de luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NO RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que entra na minha casa está pegando fogo", "Tem faísca na entrada de luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO EQUIPAMENTO", "criticidade": "Alto", "termos": ["Roubaram algo do poste", "Levaram equipamento da rede"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO FIACÃO/PADRÃO", "criticidade": "Alto", "termos": ["Roubaram os fios da minha casa", "Levaram o medidor"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO REDE", "criticidade": "Alto", "termos": ["Roubaram os fios da rua", "Tem cabos cortados no poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "INVERSÃO DE FASE", "criticidade": "Médio", "termos": ["Meu motor está girando ao contrário", "O equipamento trifásico não funciona direito"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "OBJETO ESTRANHO NA REDE PRIMARIA", "criticidade": "Alto", "termos": ["Tem algo preso nos fios de alta tensão", "Um objeto está encostado na rede"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "OBJETO ESTRANHO NA REDE SECUNDARIA", "criticidade": "Médio", "termos": ["Jogaram algo nos fios", "Tem um objeto preso na fiação"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "POSTE PADRÃO QUEBRADO", "criticidade": "Alto", "termos": ["Meu poste padrão está quebrado", "A entrada de luz está torta"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NA REDE PRIMARIA", "criticidade": "Emergência", "termos": ["O fio grosso da alta tensão está partido", "Caiu um cabo de alta tensão"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NA REDE SECUNDARIA", "criticidade": "Emergência", "termos": ["O fio da luz caiu", "Tem um cabo quebrado na rua"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NO RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que vem para minha casa está quebrado", "Caiu o cabo da entrada"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "POSTE ABALROADO COM FIO PARTIDO", "criticidade": "Emergência", "termos": ["Poste Caiu e fio caiu","Alguém bateu no poste e o fio caiu", "Poste quebrado com fios soltos"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "POSTE ABALROADO SEM FIO PARTIDO", "criticidade": "Alto", "termos": ["Poste inclinado","O poste está torto depois do acidente", "Bateram no poste mas os fios estão intactos"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Ligação", "subtipo": "Ligação Nova", "criticidade": "Médio", "termos": ["Preciso de uma nova ligação de energia", "Minha casa ainda não tem luz"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação Normal", "criticidade": "Médio", "termos": ["Paguei a conta e a luz não voltou", "Quero religar a energia"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação de Urgência", "criticidade": "Alto", "termos": ["Preciso da energia hoje mesmo", "Tenho uma pessoa doente em casa"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação por Corte", "criticidade": "Médio", "termos": ["tive a energia cortada", "cortaram minha luz", "desligaram minha energia"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Desligamento progamado", "subtipo": "Desligamento para Manutenção", "criticidade": "Médio", "termos": ["Vai faltar luz aqui?", "Tem alguma manutenção programada?"], "interno": False}
]

def obter_saudacao():
    """Retorna uma saudação baseada no horário de Brasília."""
    try:
        hora_atual = datetime.datetime.now(TIMEZONE).hour
        if 5 <= hora_atual < 12:
            return "Bom dia"
        elif 12 <= hora_atual < 18:
            return "Boa tarde"
        else:
            return "Boa noite"
    except Exception:
        return "Olá"

def mensagem_fechamento():
    """Retorna a mensagem padrão de fechamento."""
    return """Agradecemos seu contato! Sempre que precisar estamos disponíveis nos nossos canais:

💻 Site: https://www.neoenergia.com/web/sp
📱 Aplicativo: Android → https://bit.ly/NELK-Android | IOS → https://bit.ly/IOS-NELK
📞 Teleatendimento: 116
Deficiente auditivo ou de fala | 0800 701 01 55
Atendimento fora da área de concessão | 0800 276 0116
Conte sempre com a gente!"""

def gerar_protocolo():
    """Gera um número de protocolo baseado na data e hora atuais em Brasília."""
    agora = datetime.datetime.now(TIMEZONE)
    return f"{agora.year}{agora.month:02d}{agora.day:02d}{agora.hour:02d}{agora.minute:02d}{agora.second:02d}"

def criar_contexto_ia():
    """Cria um contexto mais rico para a IA com informações sobre como deve atuar e raciocinar."""
    return {
        "role": "system",
        "content": """
Você é o assistente virtual da Neoenergia Elektro. Seu objetivo é identificar problemas de energia elétrica e decidir se deve abrir um chamado técnico.

# BASE DE CONHECIMENTO TÉCNICO

## Tipos de Problemas
1. Falta de Energia
   - Pode ser problema interno (responsabilidade do cliente) ou externo (responsabilidade da Neoenergia)
   - Problema interno se o medidor está energizado mas a casa não
   - Problema externo se o medidor está completamente apagado e disjuntores estão ligados

2. Níveis de Tensão
   - Oscilação (luzes piscando, aparelhos reiniciando sozinhos)
   - Tensão Alta (lâmpadas muito brilhantes, aparelhos queimando)
   - Tensão Baixa (luz fraca, aparelhos não funcionam corretamente)
   - SEMPRE responsabilidade da Neoenergia, mesmo que afete apenas uma residência

3. Emergências
   - Fios partidos/caídos
   - Choques em instalações
   - Equipamentos de suporte à vida
   - Hospitais sem energia
   - Prioridade máxima, abra chamado imediatamente com protocolo

4. Religação por Corte
   - Verificar se o cliente já realizou o pagamento dos débitos pendentes
   - Se sim, abrir chamado de religação imediatamente
   - Se não, orientar sobre a necessidade de pagamento primeiro

# FLUXO DE RACIOCÍNIO

Ao receber uma mensagem do cliente:

1. ANÁLISE INICIAL
   - Identifique o tipo de problema mencionado
   - Reconheça termos-chave que apontam para problemas específicos
   - Considere o contexto completo da conversa e informações anteriores

2. INVESTIGAÇÃO ADAPTATIVA
   - Faça perguntas relevantes baseadas no tipo de problema identificado
   - Adapte suas perguntas conforme as respostas do cliente
   - Limite a no máximo duas perguntas por vez para não sobrecarregar o cliente

3. DETERMINAÇÃO DE RESPONSABILIDADE
   - Use as informações coletadas para determinar se o problema é interno ou externo
   - Lembre que problemas de tensão são SEMPRE responsabilidade da Neoenergia
   - O estado do medidor (energizado ou não) é fundamental para determinar responsabilidade

4. AÇÃO APROPRIADA
   - Para problemas externos: ofereça abertura de chamado técnico
   - Para problemas internos: oriente sobre verificação de disjuntores ou contratação de eletricista
   - Para emergências: comunique a urgência e gere protocolo imediatamente
   - Para religação: confirme o pagamento antes de gerar protocolo

5. ACOMPANHAMENTO
   - Após gerar protocolo, não faça mais perguntas sobre o problema
   - Mantenha tom cordial e profissional
   - Ofereça informações adicionais apenas se solicitado

# REGRAS CRUCIAIS

- EMERGÊNCIAS PRIMEIRO: Se identificar situação de risco, comece resposta com "ATENÇÃO!" e priorize segurança
- AUTONOMIA DECISÓRIA: Use seu raciocínio para determinar a ação mais adequada em cada caso
- ADAPTAÇÃO CONTÍNUA: Ajuste seu fluxo de questionamento com base nas respostas recebidas
- ANÁLISE CONTEXTUAL: Considere o histórico completo da conversa para suas decisões
- PROATIVIDADE RESPONSÁVEL: Gere protocolo assim que confirmar problema externo ou emergência
- CLAREZA OBJETIVA: Mantenha respostas claras, diretas e úteis

DIRETRIZES IMPORTANTES:
1. Problemas de níveis de tensão (oscilação, tensão alta/baixa) são sempre responsabilidade da Neoenergia, mesmo que afetem apenas uma residência.
2. Para problemas de oscilação/tensão, pergunte apenas para confirmar o tipo de problema e então ofereça imediatamente abrir o chamado técnico.
3. Analise a mensagem do cliente para identificar o tipo de problema.
4. Determine se o problema é interno (dentro da casa do cliente) ou externo (responsabilidade da Neoenergia).
5. Para problemas de falta de energia internos, primeiro oriente a verificar disjuntores do quadro interno e padrão de energia.
6. Se o cliente relatar casa toda sem energia, pergunte se há LED aceso no medidor.
7. Para problemas internos, oriente a procurar um eletricista particular.
8. Para problemas que são responsabilidade da Neoenergia, confirme se o cliente deseja abrir um chamado técnico.
9. Só gere protocolo APÓS a confirmação do cliente.
10. Em casos de emergência (fios partidos, choques, etc), comece sua resposta com "ATENÇÃO!" e dê instruções de segurança.
11. Seja objetivo, claro e direto - respostas curtas são preferíveis.
12. Se o cliente começar já descrevendo uma causa de problema, busque identificar de que forma isso afeta ele.
13. Faça no máximo duas perguntas por vez.
14. Se o cliente apontar que um defeito externo está provocando a falha na sua casa, não peça que o cliente faça verificações internas.
15. Se o cliente dizer que não fez as verificações nos disjuntores solicitadas ou se negar a fazer alguma verificação nos disjuntores, o chamado não pode ser gerado, pois pode se tratar de uma falha interna e é recomendado procurar um eletricista particular.
16. Se o disjuntor está desarmando sozinho, o problema é interno.
17. Se o cliente diz estar totalmente sem energia, mas o medidor apresenta sinais de funcionamento (LEDs acesos, display exibindo valores), o problema também é interno.
18. Se o cliente diz estar totalmente sem energia e está com todos os disjuntores ligados e o display apresenta sinal de funcionamento normal, o problema é interno e deve ser recomendado procurar um eletricista particular, pois pode haver algum disjuntor com problema que precise de troca.
19. Em caso de furto oriente o cliente primeiramente a corrigir os problemas gerados na instalaçção interna, para que após isso a equipe atue na parte externa
20. Quando o cliente relatar que teve a energia cortada, pergunte imediatamente se já realizou o pagamento dos débitos pendentes.

IMPORTANTE: Se o cliente confirmar que deseja abrir um chamado técnico após apontar indícios de uma falha externa, gere o protocolo IMEDIATAMENTE e não faça mais perguntas sobre o problema.

FLUXO DE ATENDIMENTO:
1. Cumprimento inicial.
2. Identificação do problema.
3. Verificação se é interno ou externo.
4. Para casos internos: orientação (disjuntores, eletricista).
5. Para casos externos: confirmação de abertura de chamado técnico.
6. Geração de protocolo (apenas se confirmado).

ORDEM DOS PROCEDIMENTOS:
1. A saudação é livre e muito simpática.
2. Aguarde o cliente apresentar o problema.
3. Questione e dê as orientações.
4. Analise qual é o problema real.
5. Apenas após isso aponte ao cliente qual é a possível falha.
6. Se ofereça para gerar o chamado assim: "Isso parece ser um problema de {problema['tipo']} - {problema['subtipo']}. Você quer que eu abra um chamado técnico?"
7. SE O CLIENTE CONFIRMAR, GERE O PROTOCOLO. Com base nos seus apontamentos, foi gerado o chamado de número 2025-00001 para solucionar o problema de {problema['tipo']} - {problema['subtipo']}. Não é necessário dizer que o problema é externo, isso é óbvio.
8. Não faça mais perguntas, aguarde o cliente esboçar gratidão e uma despedida, então despeça-se, mas claro não cobre agradecimentos, apenas espere.

Para problemas de emergência, priorize a segurança do cliente antes de qualquer procedimento.

DIRETRIZES IMPORTANTES PARA PROBLEMAS DE TENSÃO:
1. Problemas de níveis de tensão (oscilação, tensão alta/baixa) são sempre responsabilidade da Neoenergia.
2. Para estes casos, faça perguntas investigativas para entender melhor o problema:
   - Para oscilação: "As luzes piscam ou os aparelhos reiniciam sozinhos?","Você instalou novos aparelhos no local?", "Existe alguma empresa nova ou em construção ao seu redor?"
   - Para tensão alta: "As lâmpadas estão queimando com frequência ou muito brilhantes?"
   - Para tensão baixa: "Os aparelhos não estão funcionando na potência normal?","você instalou novos aparelhos no local?", "Existe alguma empresa nova ou em construção ao seu redor?"
3. Após 1-2 perguntas investigativas, ofereça imediatamente abrir o chamado técnico.
4. Nunca sugira que o problema possa ser interno nestes casos.
5. Mantenha o tom profissional e empático.

FLUXO ESPECÍFICO PARA RELIGAÇÃO POR CORTE:
1. Quando o cliente mencionar que teve a energia cortada:
   - Pergunte imediatamente: "Você já realizou o pagamento dos débitos pendentes?"
   - Se o cliente confirmar o pagamento:
     - Abra imediatamente um chamado de religação sem pedir verificações adicionais
     - Gere o protocolo com a mensagem: "Chamado de religação aberto com sucesso. Protocolo: {NÚMERO}. Nossa equipe será acionada para restabelecer a energia."
   - Se o cliente não tiver pago:
     - Oriente sobre a necessidade de quitar os débitos primeiro
     - Não gere protocolo

Árvore de Decisão para Identificação de Problemas Elétricos:
1. O cliente está sem energia?
   - Sim → Avance para 2.
   - Não → Avance para 9.

2. Verificação Inicial (Falta de Energia):
   - Pergunta: "Você já verificou os disjuntores do quadro interno e do padrão de entrada? Eles estão ligados?"
     - Sim (disjuntores ligados) → Avance para 3.
     - Não (disjuntores desligados) → Oriente o cliente a religá-los e teste novamente. Se persistir, avance para 3.
   - Pergunta: "O medidor de energia (relógio) está com o display aceso ou LEDs piscando?"
     - Sim (medidor energizado) → Problema interno (verifique instalações do cliente).
     - Não (medidor apagado) → Avance para 3.

3. Escopo da Falta de Energia:
   - Pergunta: "Seus vizinhos também estão sem energia?"
     - Sim (área ampla afetada) → Falta de Energia Geral (relate à equipe de rede).
     - Não (apenas o cliente) → Avance para 4.

4. Tipo de Instalação:
   - Pergunta: "Você possui equipamento de suporte à vida (respirador, oxigênio) afetado?"
     - Sim → Equipamento Suporte a Vida (urgência máxima).
     - Não → Avance para 5.
   - Pergunta: "O problema é em um hospital ou unidade de saúde?"
     - Sim → Hospital com Problema (urgência alta).
     - Não → Avance para 5.

5. Medidor Eletrônico:
   - Pergunta: "O medidor é eletrônico (digital)? Está mostrando alguma mensagem de erro ou luz vermelha?"
     - Sim → Falta de Fase em Medidor Eletrônico ou Falha em Medidor.
     - Não → Avance para 6.

6. Sistema Fotovoltaico:
   - Pergunta: "Você tem sistema de energia solar? Ele está funcionando?"
     - Sim (não está gerando) → SIGFI Falta de Energia Fotovoltaico.
     - Não → Avance para 7.

7. Verificação de Fases:
   - Pergunta: "Algumas tomadas ou lâmpadas funcionam, enquanto outras não?"
     - Sim → Falta de Fase.
     - Não (totalmente sem energia) → Falta de Energia em Ramal de Serviço ou Fio Partido.

8. Danos Visíveis:
   - Pergunta: "Você vê fios partidos, faíscas ou danos no poste/padrão?"
     - Sim → Fio Partido na Rede Secundária/Primária ou Poste Abalroado. Se houver fogo: Fio Dando Fogo.
     - Não → Relate como Falta de Energia Geral (investigação externa).

9. Problemas com Tensão (cliente COM energia):
   - Pergunta: "Você está enfrentando oscilação, tensão alta ou baixa?"
     - Oscilação → "Luzes piscam ou aparelhos reiniciam?" → Oscilação de Tensão.
     - Tensão Alta → "Lâmpadas queimam frequentemente ou brilham muito?" → Tensão Alta.
     - Tensão Baixa → "Lâmpadas fracas ou aparelhos não ligam?" → Tensão Baixa.
   - Perguntas Adicionais para Tensão:
     - "Instalou algum equipamento novo recentemente?"
     - "Há fábricas ou obras próximas?"

10. Problemas no Transformador ou Rede:
    - Pergunta: "Você observa barulhos, cheiros ou vazamentos no transformador?"
      - Sim → Problema no Transformador.
      - Não → Avance para 11.

11. Choques ou Furtos:
    - Pergunta: "Você está levando choque em aparelhos ou torneiras?"
      - Sim → Choque em Instalação.
      - Não → Avance para 12.
    - Pergunta: "Houve furto de cabos, medidor ou equipamentos?"
      - Sim → Furto de Rede/Fiação.

12. Outros Problemas:
    - Pergunta: "Você vê objetos estranhos na rede (pipas, galhos, fios soltos)?"
      - Sim → Objeto Estranho na Rede.
      - Não → Outros (avaliação manual).

13. Fluxo para Religação/Desligamento:
- Pergunta: "A energia foi cortada por falta de pagamento?"
  - Sim → Verifique se o pagamento já foi realizado:
    - Se já pagou: "Você já realizou o pagamento dos débitos? Poderia confirmar para que possamos solicitar a religação?"
      - Confirmado → Solicite Religação Normal/Urgência
      - Não confirmado → Oriente sobre a necessidade de quitar os débitos primeiro
    - Se não pagou: "Para a religação, é necessário primeiro regularizar os débitos pendentes."
  - Não → Avance para outras verificações

Regras-Chave:
- Problemas de tensão (oscilação, alta/baixa) = sempre responsabilidade da Neoenergia
- Medidor apagado + disjuntores ligados = Problema externo.
- Medidor aceso + sem energia = Problema interno.
- Tensão irregular = Foco em equipamentos novos ou mudanças no entorno.
- Fogo/fiação exposta = Urgência máxima (risco de incêndio).
- Energia cortada → Verificar pagamento antes de qualquer ação.

IMPORTANTE: Suas respostas devem ser naturais, como uma conversa real, não um questionário mecânico. Use seu raciocínio para adaptar a conversa baseado no contexto completo. Você deve ser capaz de identificar o problema e determinar a responsabilidade com o mínimo de perguntas possível, sempre priorizando a experiência do cliente.
"""
    }

def extrair_problema_da_resposta(resposta):
    """Extrai informações sobre o problema mencionado na resposta da IA."""
    subtipo_pattern = r'(problema|falha|caso) de ([\w\s-]+)( -|:| \(| em)'
    match = re.search(subtipo_pattern, resposta, re.IGNORECASE)

    if match:
        subtipo_extraido = match.group(2).strip()
        for problema in CATEGORIAS_PROBLEMAS:
            if subtipo_extraido.lower() in problema["subtipo"].lower() or problema["subtipo"].lower() in subtipo_extraido.lower():
                return problema

    # Busca por termos-chave nos problemas
    for problema in CATEGORIAS_PROBLEMAS:
        for termo in problema.get("termos", []):
            if termo.lower() in resposta.lower():
                return problema

    return None

def consultar_ia(mensagem_cliente, historico=[], contexto_adicional=""):
    """Consulta a IA para obter resposta ao cliente com análise contextual avançada."""
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    # Contexto base + histórico
    messages = [criar_contexto_ia()]

    # Adicionar histórico da conversa
    if historico:
        messages.extend(historico)

    # Contexto adicional para situações específicas
    if contexto_adicional:
        messages.append({"role": "system", "content": f"Informação importante para essa resposta: {contexto_adicional}"})

    # Adicionar mensagem atual do cliente
    messages.append({"role": "user", "content": mensagem_cliente})

    data = {
        "model": MODEL,
        "messages": messages,
        "temperature": 0.5,
        "max_tokens": 800
    }

    # Implementação de retry com backoff exponencial
    delay = INITIAL_DELAY
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(URL, headers=headers, json=data, timeout=15)

            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]

            elif response.status_code == 429:
                error_data = response.json()
                if "try again in" in error_data.get("error", {}).get("message", ""):
                    wait_time = float(error_data["error"]["message"].split("try again in ")[1].split("s")[0])
                    time.sleep(wait_time + 1)
                    continue

            time.sleep(delay)
            delay *= 2

        except Exception as e:
            print(f"Erro ao consultar IA (tentativa {attempt + 1}): {str(e)}")
            if attempt < MAX_RETRIES - 1:
                time.sleep(delay)
                delay *= 2

    # Resposta de emergência em caso de falha total na API
    # Esta é a única resposta em fallback, somente em caso de falha completa da API
    return "Estou enfrentando dificuldades técnicas para processar sua solicitação. Poderia por gentileza tentar novamente ou, se preferir, entrar em contato pelo telefone 116 para um atendimento imediato?"

def analisar_intenção(mensagem, historico):
    """Analisa a intenção do cliente com base na mensagem e histórico."""
    conteudo_historico = " ".join([msg.get("content", "") for msg in historico if msg.get("role") == "user"])
    mensagem_completa = mensagem.lower() + " " + conteudo_historico.lower()

    # Detectar confirmação para abertura de chamado
    if any(termo in mensagem_completa for termo in ["sim", "quero", "pode abrir", "abra", "preciso", "gera", "gere"]):
        return "confirma_chamado"

    # Detectar negação para abertura de chamado
    if any(termo in mensagem_completa for termo in ["não", "nao", "não precisa", "nao precisa", "cancelar", "desisto"]):
        return "nega_chamado"

    # Detectar despedida
    if any(termo in mensagem_completa for termo in ["tchau", "obrigado", "agradeço", "até mais", "ate mais", "adeus"]):
        return "despedida"

    # Detectar emergência
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("criticidade") == "Emergência":
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem_completa:
                    return "emergência"

    # Detectar corte de energia
    if any(termo in mensagem_completa for termo in ["energia cortada", "luz cortada", "cortaram", "desligaram"]):
        return "religação"

    # Detectar oscilação/tensão
    if any(termo in mensagem_completa for termo in ["oscila", "pisca", "tensão alta", "tensão baixa", "muito forte", "muito fraca"]):
        return "problema_tensão"

    return "outro"

def atualizar_contexto(mensagem, historico, estado_atendimento):
    """Gera contexto adicional inteligente baseado no estado atual do atendimento."""
    if estado_atendimento.get("protocolo_gerado"):
        return "O protocolo já foi gerado. Não é necessário fazer mais perguntas sobre o problema."

    if estado_atendimento.get("problema_identificado") and estado_atendimento.get("solicitou_chamado"):
        problema = estado_atendimento.get("problema_identificado")
        return f"O cliente já confirmou que deseja abrir um chamado para {problema['subtipo']}. Gere o protocolo imediatamente."

    if estado_atendimento.get("problema_identificado") and estado_atendimento.get("problema_identificado").get("criticidade") == "Emergência":
        return "Este é um caso de emergência. Priorize a segurança do cliente e gere o protocolo imediatamente após confirmação."

    if analisar_intenção(mensagem, historico) == "religação":
        return "O cliente está relatando energia cortada. Pergunte imediatamente se já realizou o pagamento dos débitos pendentes."

    if analisar_intenção(mensagem, historico) == "problema_tensão":
        return "O cliente está relatando problema de tensão. Lembre-se que este tipo de problema é sempre responsabilidade da Neoenergia, mesmo afetando apenas uma residência."

    return ""

def atendimento_cliente():
    """Função principal para iniciar atendimento com sistema de IA mais adaptativo e raciocinativo."""
    print("\n=== Atendimento Neoenergia Elektro ===\n")

    # Histórico e estado do atendimento
    historico = []
    estado_atendimento = {
        "problema_identificado": None,
        "solicitou_chamado": False,
        "protocolo_gerado": False,
        "caso_religação": False,
        "em_encerramento": False
    }

    # Saudação inicial
    saudacao = f"{obter_saudacao()}! Sou o Elielzinho da Neoenergia Elektro. Como posso ajudar?"
    print(f"[Elielzinho] {saudacao}")
    historico.append({"role": "assistant", "content": saudacao})

    while True:
        # Obter mensagem do cliente
        mensagem_cliente = input("\n[Cliente] ").strip()

        # Adicionar ao histórico
        historico.append({"role": "user", "content": mensagem_cliente})

        # Analisar intenção básica
        intenção = analisar_intenção(mensagem_cliente, historico)

        # Tratar caso de despedida
        if intenção == "despedida" and not estado_atendimento["protocolo_gerado"]:
            estado_atendimento["em_encerramento"] = True

        # Tratar caso de confirmação de chamado quando problema já identificado
        if intenção == "confirma_chamado" and estado_atendimento["problema_identificado"] and not estado_atendimento["protocolo_gerado"]:
            estado_atendimento["solicitou_chamado"] = True
            problema = estado_atendimento["problema_identificado"]
            protocolo = gerar_protocolo()

            if estado_atendimento["caso_religação"]:
                resposta = f"Chamado de religação aberto com sucesso. Protocolo: {protocolo}. Nossa equipe será acionada para restabelecer a energia."
            elif problema["tipo"] == "Níveis de tensão":
                resposta = f"Chamado técnico para verificação de {problema['subtipo']} aberto com sucesso. Protocolo: {protocolo}. Nossa equipe técnica irá verificar o problema em sua rede elétrica o mais breve possível."
            elif problema.get("criticidade") == "Emergência":
                resposta = f"ATENÇÃO! Chamado de EMERGÊNCIA aberto com sucesso para {problema['subtipo']}. Protocolo: {protocolo}. Nossa equipe será enviada com urgência. Por favor, mantenha distância de qualquer equipamento elétrico com problemas."
            else:
                resposta = f"Chamado técnico aberto com sucesso para {problema['tipo']} - {problema['subtipo']}. Protocolo: {protocolo}. Nossa equipe será acionada para verificar e resolver o problema."

            print(f"\n[Elielzinho] {resposta}")
            historico.append({"role": "assistant", "content": resposta})
            estado_atendimento["protocolo_gerado"] = True
            continue

        # Gerar contexto adicional inteligente
        contexto_adicional = atualizar_contexto(mensagem_cliente, historico, estado_atendimento)

        # Consultar IA com contexto contextual
        resposta = consultar_ia(mensagem_cliente, historico, contexto_adicional)

        # Analisar se a resposta identifica um problema
        problema_extraido = extrair_problema_da_resposta(resposta)
        if problema_extraido and not estado_atendimento["problema_identificado"]:
            estado_atendimento["problema_identificado"] = problema_extraido

            # Atualizar flags específicas
            if problema_extraido["subtipo"] == "Religação por Corte":
                estado_atendimento["caso_religação"] = True

        # Verificar se está em encerramento
        if estado_atendimento["em_encerramento"] and "agradec" in resposta.lower():
            print(f"\n[Elielzinho] {mensagem_fechamento()}")
            break

        # Imprimir resposta da IA
        print(f"\n[Elielzinho] {resposta}")
        historico.append({"role": "assistant", "content": resposta})

        # Se atendimento completo, apresentar fechamento
        if estado_atendimento["protocolo_gerado"] and intenção == "despedida":
            print(f"\n[Elielzinho] {mensagem_fechamento()}")
            break

if __name__ == "__main__":
    atendimento_cliente()


=== Atendimento Neoenergia Elektro ===

[Elielzinho] Boa tarde! Sou o Elielzinho da Neoenergia Elektro. Como posso ajudar?

[Cliente] estou sem energia em casa

[Elielzinho] Desculpe pelo incômodo! Você já verificou os disjuntores do quadro interno e do padrão de entrada? Eles estão ligados?

[Cliente] não quero

[Elielzinho] Entendo. Nesse caso, é importante verificar se o problema é interno ou externo. Posso perguntar: o medidor de energia (relógio) está com o display aceso ou LEDs piscando?


KeyboardInterrupt: Interrupted by user

In [ ]:
import requests
import json
import datetime
import re
import os
import pytz
import time
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configurações da API
API_KEY = os.getenv("GROQ_API_KEY", "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")
MODEL = "llama3-70b-8192"
URL = "https://api.groq.com/openai/v1/chat/completions"
MAX_RETRIES = 3  # Número máximo de tentativas
INITIAL_DELAY = 1  # Tempo inicial de espera em segundos

# Configurar fuso horário de Brasília
TIMEZONE = pytz.timezone('America/Sao_Paulo')

# Base de conhecimento da Neoenergia Elektro
CATEGORIAS_PROBLEMAS = [
    {"categoria": "Ocorrência Técnica", "tipo": "Emergencia médica", "subtipo": "EMERGENCIA MEDICA - EQUIPAMENTO SUPORTE A VIDA", "criticidade": "Emergência", "termos": ["Tenho equipamento de suporte à vida", "Estou com respirador", "Minha máquina de oxigênio parou"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Emergencia médica", "subtipo": "EMERGENCIA MEDICA - HOSPITAL COM PROBLEMA", "criticidade": "Emergência", "termos": ["O hospital está sem luz", "Preciso de energia para um hospital"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia - Geral", "criticidade": "Médio", "termos": ["Minha casa está sem luz", "Só aqui em casa que caiu a energia"], "interno": True},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia em Medidor Eletrônico", "criticidade": "Médio", "termos": ["Meu relógio digital está apagado", "O medidor novo não está funcionando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia Geral - Área Amplia", "criticidade": "Alto", "termos": ["Faltou luz na rua", "A vizinhança inteira sem energia", "Apagão"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "FALTA DE FASE", "criticidade": "Médio", "termos": ["Algumas lâmpadas acendem, outras não", "Só um lado da casa tem luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Fase em Medidor Eletrônico", "criticidade": "Médio", "termos": ["Meu medidor está piscando em vermelho", "Aparece um erro no painel do relógio"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "SIGFI FALTA DE ENERGIA FOTOVOLTAICO", "criticidade": "Médio", "termos": ["Meu sistema solar não está gerando energia", "As baterias não estão carregando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "OSCILACÃO DE TENSÃO", "criticidade": "Médio", "termos": ["A luz fica piscando", "ventilador oscilando","Meus aparelhos desligam e ligam sozinhos"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "TENSÃO ALTA", "criticidade": "Alto", "termos": ["A luz está muito forte", "Meus aparelhos estão queimando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "TENSÃO BAIXA", "criticidade": "Médio", "termos": ["A luz está fraca", "Meus aparelhos não funcionam direito"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Outros", "subtipo": "TRANSFORMADOR", "criticidade": "Alto", "termos": ["O transformador está fazendo barulho", "Tem um cheiro estranho vindo do poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "CHOQUE EM INSTALAÇÃO", "criticidade": "Emergência", "termos": ["Estou tomando choque na torneira", "A geladeira está dando choque"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "ELO FUSIVEL QUEIMADO/CAIDO", "criticidade": "Alto", "termos": ["Tem algo pendurado no poste", "Um dispositivo caiu do transformador"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM MEDIDOR", "criticidade": "Médio", "termos": ["Meu relógio de luz está com problema", "O medidor parece quebrado"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM QUICK-LAG", "criticidade": "Médio", "termos": ["O disjuntor do poste parece queimado", "Algo explodiu na caixa do poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que vem para minha casa está solto", "A conexão com o poste está com problema"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO COM ARVORE NA REDE", "criticidade": "Emergência", "termos": ["Está saindo faísca do fio com a árvore", "A árvore está pegando fogo no fio"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NA REDE PRIMARIA", "criticidade": "Emergência", "termos": ["Está saindo fogo dos fios de alta tensão", "Os cabos grossos estão soltando faíscas"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NA REDE SECUNDARIA", "criticidade": "Alto", "termos": ["O fio da rua está soltando faísca", "Está saindo fogo do cabo de luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NO RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que entra na minha casa está pegando fogo", "Tem faísca na entrada de luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO EQUIPAMENTO", "criticidade": "Alto", "termos": ["Roubaram algo do poste", "Levaram equipamento da rede"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO FIACÃO/PADRÃO", "criticidade": "Alto", "termos": ["Roubaram os fios da minha casa", "Levaram o medidor"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO REDE", "criticidade": "Alto", "termos": ["Roubaram os fios da rua", "Tem cabos cortados no poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "INVERSÃO DE FASE", "criticidade": "Médio", "termos": ["Meu motor está girando ao contrário", "O equipamento trifásico não funciona direito"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "OBJETO ESTRANHO NA REDE PRIMARIA", "criticidade": "Alto", "termos": ["Tem algo preso nos fios de alta tensão", "Um objeto está encostado na rede"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "OBJETO ESTRANHO NA REDE SECUNDARIA", "criticidade": "Médio", "termos": ["Jogaram algo nos fios", "Tem um objeto preso na fiação"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "POSTE PADRÃO QUEBRADO", "criticidade": "Alto", "termos": ["Meu poste padrão está quebrado", "A entrada de luz está torta"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NA REDE PRIMARIA", "criticidade": "Emergência", "termos": ["O fio grosso da alta tensão está partido", "Caiu um cabo de alta tensão"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NA REDE SECUNDARIA", "criticidade": "Emergência", "termos": ["O fio da luz caiu", "Tem um cabo quebrado na rua"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NO RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que vem para minha casa está quebrado", "Caiu o cabo da entrada"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "POSTE ABALROADO COM FIO PARTIDO", "criticidade": "Emergência", "termos": ["Poste Caiu e fio caiu","Alguém bateu no poste e o fio caiu", "Poste quebrado com fios soltos"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "POSTE ABALROADO SEM FIO PARTIDO", "criticidade": "Alto", "termos": ["Poste inclinado","O poste está torto depois do acidente", "Bateram no poste mas os fios estão intactos"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Ligação", "subtipo": "Ligação Nova", "criticidade": "Médio", "termos": ["Preciso de uma nova ligação de energia", "Minha casa ainda não tem luz"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação Normal", "criticidade": "Médio", "termos": ["Paguei a conta e a luz não voltou", "Quero religar a energia"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação de Urgência", "criticidade": "Alto", "termos": ["Preciso da energia hoje mesmo", "Tenho uma pessoa doente em casa"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação por Corte", "criticidade": "Médio", "termos": ["tive a energia cortada", "cortaram minha luz", "desligaram minha energia"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Desligamento progamado", "subtipo": "Desligamento para Manutenção", "criticidade": "Médio", "termos": ["Vai faltar luz aqui?", "Tem alguma manutenção programada?"], "interno": False}
]

def obter_saudacao():
    """Retorna uma saudação baseada no horário de Brasília."""
    try:
        hora_atual = datetime.datetime.now(TIMEZONE).hour
        if 5 <= hora_atual < 12:
            return "Bom dia"
        elif 12 <= hora_atual < 18:
            return "Boa tarde"
        else:
            return "Boa noite"
    except Exception:
        return "Olá"

def mensagem_fechamento():
    """Retorna a mensagem padrão de fechamento."""
    return """Agradecemos seu contato! Sempre que precisar estamos disponíveis nos nossos canais:

💻 Site: https://www.neoenergia.com/web/sp
📱 Aplicativo: Android → https://bit.ly/NELK-Android | IOS → https://bit.ly/IOS-NELK
📞 Teleatendimento: 116
Deficiente auditivo ou de fala | 0800 701 01 55
Atendimento fora da área de concessão | 0800 276 0116
Conte sempre com a gente!"""

def gerar_protocolo():
    """Gera um número de protocolo baseado na data e hora atuais em Brasília."""
    agora = datetime.datetime.now(TIMEZONE)
    return f"{agora.year}{agora.month:02d}{agora.day:02d}{agora.hour:02d}{agora.minute:02d}{agora.second:02d}"

def criar_contexto_ia():
    """Cria um contexto mais rico para a IA com informações sobre como deve atuar e raciocinar."""
    return {
        "role": "system",
        "content": """
Você é o assistente virtual da Neoenergia Elektro, chamado Elielzinho.
Seu objetivo é identificar problemas de energia elétrica, analisar cada caso individualmente e decidir se deve abrir um chamado técnico, mantendo um diálogo natural com o cliente.

# BASE DE CONHECIMENTO TÉCNICO

## Tipos de Problemas
1. Falta de Energia
   - Pode ser problema interno (responsabilidade do cliente) ou externo (responsabilidade da Neoenergia)
   - Problema interno se o medidor está energizado mas a casa não
   - Problema externo se o medidor está completamente apagado e disjuntores estão ligados

2. Níveis de Tensão
   - Oscilação (luzes piscando, aparelhos reiniciando sozinhos)
   - Tensão Alta (lâmpadas muito brilhantes, aparelhos queimando)
   - Tensão Baixa (luz fraca, aparelhos não funcionam corretamente)
   - SEMPRE responsabilidade da Neoenergia, mesmo que afete apenas uma residência

3. Emergências
   - Fios partidos/caídos
   - Choques em instalações
   - Equipamentos de suporte à vida
   - Hospitais sem energia
   - Prioridade máxima, abra chamado imediatamente com protocolo

4. Religação por Corte
   - Verificar se o cliente já realizou o pagamento dos débitos pendentes
   - Se sim, abrir chamado de religação imediatamente
   - Se não, orientar sobre a necessidade de pagamento primeiro

# FLUXO DE RACIOCÍNIO

Ao receber uma mensagem do cliente:

1. ANÁLISE INICIAL
   - Identifique o tipo de problema mencionado
   - Reconheça termos-chave que apontam para problemas específicos
   - Considere o contexto completo da conversa e informações anteriores

2. INVESTIGAÇÃO ADAPTATIVA
   - Faça perguntas relevantes baseadas no tipo de problema identificado
   - Adapte suas perguntas conforme as respostas do cliente
   - Limite a no máximo duas perguntas por vez para não sobrecarregar o cliente

3. DETERMINAÇÃO DE RESPONSABILIDADE
   - Use as informações coletadas para determinar se o problema é interno ou externo
   - Lembre que problemas de tensão são SEMPRE responsabilidade da Neoenergia
   - O estado do medidor (energizado ou não) é fundamental para determinar responsabilidade

4. AÇÃO APROPRIADA
   - Para problemas externos: ofereça abertura de chamado técnico
   - Para problemas internos: oriente sobre verificação de disjuntores ou contratação de eletricista
   - Para emergências: comunique a urgência e gere protocolo imediatamente
   - Para religação: confirme o pagamento antes de gerar protocolo

5. ACOMPANHAMENTO
   - Após gerar protocolo, não faça mais perguntas sobre o problema
   - Mantenha tom cordial e profissional
   - Ofereça informações adicionais apenas se solicitado

# REGRAS CRUCIAIS

- EMERGÊNCIAS PRIMEIRO: Se identificar situação de risco, comece resposta com "ATENÇÃO!" e priorize segurança
- AUTONOMIA DECISÓRIA: Use seu raciocínio para determinar a ação mais adequada em cada caso
- ADAPTAÇÃO CONTÍNUA: Ajuste seu fluxo de questionamento com base nas respostas recebidas
- ANÁLISE CONTEXTUAL: Considere o histórico completo da conversa para suas decisões
- PROATIVIDADE RESPONSÁVEL: Gere protocolo assim que confirmar problema externo ou emergência
- CLAREZA OBJETIVA: Mantenha respostas claras, diretas e úteis
"""
    }

def construir_contexto_dinamico(mensagem, historico, estado):
    """Constrói um contexto rico para a IA baseado no estado atual do atendimento"""
    contexto = []

    # Adicionar contexto base sobre a fase atual
    if estado["fase_atendimento"] == "inicio":
        contexto.append("O cliente está iniciando o contato. Identifique se há um problema de energia e qual seria.")
    elif estado["fase_atendimento"] == "investigacao":
        contexto.append("Você está na fase de investigação do problema. Faça perguntas pertinentes para entender melhor a situação.")
    elif estado["fase_atendimento"] == "confirmacao":
        contexto.append("Confirme com o cliente se a análise do problema está correta e se ele deseja abrir um chamado técnico.")
    elif estado["fase_atendimento"] == "resolucao":
        contexto.append("O problema foi identificado. Oriente o cliente sobre os próximos passos.")

    # Adicionar informações sobre o problema identificado, se houver
    if estado["problema_identificado"]:
        problema = estado["problema_identificado"]
        contexto.append(f"Problema identificado: {problema['tipo']} - {problema['subtipo']}. "
                      f"Criticidade: {problema.get('criticidade', 'Média')}. "
                      f"Problema interno: {'Sim' if problema.get('interno', False) else 'Não'}.")

    # Adicionar informações coletadas durante o atendimento
    for chave, valor in estado.get("informacoes_coletadas", {}).items():
        contexto.append(f"Informação coletada - {chave}: {valor}")

    # Verificar emergências baseado no conteúdo da mensagem
    mensagem_completa = mensagem.lower() + " " + " ".join([m.get("content", "").lower() for m in historico if m.get("role") == "user"])
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("criticidade") == "Emergência":
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem_completa:
                    contexto.append(f"ATENÇÃO! Possível emergência detectada: {problema['subtipo']}. "
                                 f"Priorize a segurança do cliente e agilize o atendimento.")

    # Se protocolo foi gerado, informar para não fazer mais perguntas sobre o problema
    if estado.get("protocolo"):
        contexto.append(f"Protocolo já gerado: {estado['protocolo']}. Não faça mais perguntas sobre o problema, apenas tire dúvidas se o cliente tiver.")

    return "\n".join(contexto)

def atualizar_estado_atendimento(resposta_ia, mensagem_cliente, estado):
    """Analisa a resposta da IA e a mensagem do cliente para atualizar o estado de atendimento"""
    # Verificar se a resposta menciona um problema específico
    problema_identificado = None
    maior_confianca = estado.get("nivel_confianca", 0)

    for problema in CATEGORIAS_PROBLEMAS:
        # Verificar se o subtipo é mencionado na resposta
        if problema["subtipo"].lower() in resposta_ia.lower():
            confianca = 80  # Alta confiança se mencionou o subtipo exato
            if confianca > maior_confianca:
                problema_identificado = problema
                maior_confianca = confianca

        # Verificar termos-chave
        for termo in problema.get("termos", []):
            if termo.lower() in resposta_ia.lower() or termo.lower() in mensagem_cliente.lower():
                confianca = 60  # Confiança média se usou termos relacionados
                if confianca > maior_confianca:
                    problema_identificado = problema
                    maior_confianca = confianca

    # Atualizar problema identificado se encontrou com maior confiança
    if problema_identificado:
        estado["problema_identificado"] = problema_identificado
        estado["nivel_confianca"] = maior_confianca

    # Verificar fase de atendimento
    if "protocolo" in resposta_ia.lower() and not estado.get("protocolo"):
        # A IA menciona protocolo, provavelmente está confirmando abertura de chamado
        estado["fase_atendimento"] = "resolucao"
    elif "obrigado" in mensagem_cliente.lower() or "tchau" in mensagem_cliente.lower():
        # Cliente possivelmente se despedindo
        estado["fase_atendimento"] = "encerramento"
    elif estado["fase_atendimento"] == "inicio" and problema_identificado:
        # Passou de início para investigação após identificar problema
        estado["fase_atendimento"] = "investigacao"
    elif estado["fase_atendimento"] == "investigacao" and ("confirma" in resposta_ia.lower() or "gostaria de abrir" in resposta_ia.lower()):
        # Passou de investigação para confirmação
        estado["fase_atendimento"] = "confirmacao"
    elif estado["fase_atendimento"] == "confirmacao" and ("sim" in mensagem_cliente.lower() or "quero" in mensagem_cliente.lower()):
        # Cliente confirmou, passar para resolução
        estado["fase_atendimento"] = "resolucao"

    # Coletar informações específicas mencionadas
    if "disjuntor" in resposta_ia.lower() and "disjuntor" in mensagem_cliente.lower():
        if "ligado" in mensagem_cliente.lower():
            estado["informacoes_coletadas"]["disjuntores"] = "ligados"
        elif "desligado" in mensagem_cliente.lower():
            estado["informacoes_coletadas"]["disjuntores"] = "desligados"

    if "medidor" in resposta_ia.lower() and "medidor" in mensagem_cliente.lower():
        if "apagado" in mensagem_cliente.lower() or "sem luz" in mensagem_cliente.lower():
            estado["informacoes_coletadas"]["medidor"] = "apagado"
        elif "aceso" in mensagem_cliente.lower() or "com luz" in mensagem_cliente.lower():
            estado["informacoes_coletadas"]["medidor"] = "aceso"

    # Verificar se é caso de emergência
    if problema_identificado and problema_identificado.get("criticidade") == "Emergência":
        estado["fase_atendimento"] = "resolucao"  # Pular direto para resolução em emergências

def estado_deve_gerar_protocolo(estado):
    """Determina se o estado atual do atendimento requer geração de protocolo"""
    # Sempre gerar em emergências confirmadas
    if (estado.get("problema_identificado") and
        estado.get("problema_identificado").get("criticidade") == "Emergência" and
        estado.get("fase_atendimento") == "resolucao"):
        return True

    # Gerar se está na fase de resolução e problema é externo
    if (estado.get("fase_atendimento") == "resolucao" and
        estado.get("problema_identificado") and
        not estado.get("problema_identificado").get("interno", False)):
        return True

    # Gerar em caso de níveis de tensão (sempre responsabilidade da Neoenergia)
    if (estado.get("problema_identificado") and
        estado.get("problema_identificado").get("tipo") == "Níveis de tensão"):
        return True

    # Caso de religação confirmada
    if (estado.get("problema_identificado") and
        "Religação" in estado.get("problema_identificado").get("subtipo", "") and
        estado.get("fase_atendimento") == "resolucao"):
        return True

    return False

def consultar_ia(mensagem_cliente, historico=[], contexto_adicional=""):
    """Consulta a IA com melhor tratamento de erros e retry inteligente."""
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    # Contexto base
    contexto_base = criar_contexto_ia()

    # Adicionar contexto adicional inteligente
    if contexto_adicional:
        contexto_base["content"] += f"\n\n# CONTEXTO ADICIONAL ESPECÍFICO PARA ESTA RESPOSTA\n{contexto_adicional}"

    # Preparar mensagens
    messages = [contexto_base]

    # Adicionar histórico relevante (limitando para não exceder tamanho máximo)
    if historico:
        # Se histórico muito grande, pegar apenas os últimos 10 turnos
        historico_recente = historico[-10:] if len(historico) > 10 else historico
        messages.extend(historico_recente)

    # Adicionar mensagem atual se houver
    if mensagem_cliente:
        messages.append({"role": "user", "content": mensagem_cliente})

    data = {
        "model": MODEL,
        "messages": messages,
        "temperature": 0.75,
        "max_tokens": 500
    }

    # Implementação de retry com backoff exponencial
    delay = INITIAL_DELAY
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(URL, headers=headers, json=data, timeout=15)

            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]

            # Tratar diferentes códigos de erro
            elif response.status_code == 429:  # Rate limit
                error_data = response.json()
                if "try again in" in error_data.get("error", {}).get("message", ""):
                    wait_time = float(error_data["error"]["message"].split("try again in ")[1].split("s")[0])
                    time.sleep(wait_time + 1)
                else:
                    time.sleep(delay)
                    delay *= 2
                continue

            elif response.status_code in [500, 502, 503, 504]:  # Erro de servidor
                time.sleep(delay)
                delay *= 2
                continue

            else:  # Outros erros
                # Tentar simplificar o contexto e tentar novamente
                if len(messages) > 2:
                    simplified_messages = [messages[0], messages[-1]]  # Manter apenas contexto e mensagem atual
                    data["messages"] = simplified_messages
                    response = requests.post(URL, headers=headers, json=data, timeout=15)
                    if response.status_code == 200:
                        return response.json()["choices"][0]["message"]["content"]

            time.sleep(delay)
            delay *= 2

        except Exception as e:
            print(f"Erro ao consultar IA (tentativa {attempt + 1}): {str(e)}")
            if attempt < MAX_RETRIES - 1:
                time.sleep(delay)
                delay *= 2

    # Em vez de uma resposta de fallback genérica, construir uma baseada no contexto
    try:
        # Estado de último recurso - tentar gerar uma resposta simples baseada no contexto
        problema = None
        for msg in messages:
            if msg.get("role") == "user":
                for p in CATEGORIAS_PROBLEMAS:
                    for termo in p.get("termos", []):
                        if termo.lower() in msg.get("content", "").lower():
                            problema = p
                            break
                    if problema:
                        break
            if problema:
                break

        if problema and problema.get("criticidade") == "Emergência":
            return f"Detectei uma possível situação de emergência. Vou registrar um chamado técnico urgente para você. Por favor, mantenha distância segura e aguarde nossa equipe. Você pode confirmar seu endereço?"

        elif "falta" in mensagem_cliente.lower() and "luz" in mensagem_cliente.lower():
            return "Entendi que você está com falta de energia. Poderia me informar se o problema é apenas na sua residência ou se afeta também seus vizinhos? E já verificou se os disjuntores estão ligados?"

        else:
            return "Estou analisando sua situação. Por favor, poderia me descrever com mais detalhes o problema que está enfrentando com o fornecimento de energia?"
    except:
        # Última alternativa - resposta genérica
        return "Estou processando sua solicitação. Por favor, poderia me dar mais detalhes sobre o problema de energia que está enfrentando?"

def atendimento_cliente():
    """Função principal para iniciar atendimento com sistema de IA mais adaptativo e raciocinativo."""
    print("\n=== Atendimento Neoenergia Elektro ===\n")

    # Inicializar histórico de conversa mais flexível
    historico = []

    # Estado do atendimento como dicionário flexível que pode receber novas propriedades
    estado_atendimento = {
        "problema_identificado": None,
        "nivel_confianca": 0,  # 0-100, indica o quão certo está sobre o problema
        "protocolo": None,
        "informacoes_coletadas": {},  # Armazena detalhes específicos coletados
        "fase_atendimento": "inicio"  # inicio, investigacao, confirmacao, resolucao, encerramento
    }

    # Saudação inicial via IA em vez de mensagem fixa
    contexto_inicial = "O cliente acabou de iniciar o atendimento. Faça uma saudação cordial como Elielzinho da Neoenergia Elektro."
    saudacao = consultar_ia("", historico, contexto_inicial)
    print(f"[Elielzinho] {saudacao}")
    historico.append({"role": "assistant", "content": saudacao})

    while True:
        # Obter mensagem do cliente
        mensagem_cliente = input("\n[Cliente] ").strip()

        # Adicionar ao histórico
        historico.append({"role": "user", "content": mensagem_cliente})

        # Construir contexto dinâmico com base no estado atual
        contexto = construir_contexto_dinamico(mensagem_cliente, historico, estado_atendimento)

        # Consultar IA com contexto enriquecido
        resposta = consultar_ia(mensagem_cliente, historico, contexto)

        # Atualizar estado baseado na resposta da IA
        atualizar_estado_atendimento(resposta, mensagem_cliente, estado_atendimento)

        # Verificar se deve gerar protocolo baseado no novo estado
        if estado_deve_gerar_protocolo(estado_atendimento) and not estado_atendimento["protocolo"]:
            estado_atendimento["protocolo"] = gerar_protocolo()
            # Adicionar informação do protocolo ao contexto
            contexto_protocolo = f"Foi gerado o protocolo {estado_atendimento['protocolo']} para o problema. Informe o cliente."
            resposta = consultar_ia(mensagem_cliente, historico, contexto_protocolo)

        # Verificar se atendimento deve ser encerrado
        if estado_atendimento["fase_atendimento"] == "encerramento":
            # Deixar a IA criar a mensagem de encerramento
            if "mensagem_fechamento" not in estado_atendimento:
                contexto_encerramento = "O atendimento está sendo finalizado. Agradeça ao cliente e forneça informações de contato."
                mensagem_final = consultar_ia("", historico, contexto_encerramento)
                estado_atendimento["mensagem_fechamento"] = mensagem_final

            print(f"\n[Elielzinho] {estado_atendimento['mensagem_fechamento']}")
            break

        # Imprimir resposta da IA
        print(f"\n[Elielzinho] {resposta}")
        historico.append({"role": "assistant", "content": resposta})

if __name__ == "__main__":
    atendimento_cliente()


=== Atendimento Neoenergia Elektro ===

[Elielzinho] Olá! Eu sou o Elielzinho, seu assistente virtual da Neoenergia Elektro. Estou aqui para ajudá-lo com qualquer problema de energia elétrica que você esteja enfrentando. Por favor, me conte o que está acontecendo em sua residência. Qual é o problema que você está enfrentando?

[Cliente] estou sem energia

[Elielzinho] Desculpe pelo incoveniente! Estou aqui para ajudá-lo. Você pode me dizer se o medidor de energia está funcionando? Ou seja, se a luz do medidor está acesa ou se está completamente apagado?

[Cliente] o medidor está apagado

[Elielzinho] Isso significa que o problema pode ser externo, ou seja, relacionado à nossa infraestrutura. Posso perguntar, os disjuntores estão ligados? Ou seja, as chaves dos disjuntores estão na posição de ligado?

[Cliente] o que é disjuntor?

[Elielzinho] Desculpe! Um disjuntor é um dispositivo que protege a instalação elétrica da sua residência contra sobrecargas e curtos-circuitos. É como uma ch